#  NOAA DSDP taxa list
## 1-96 taxa

Create list of taxa for NOAA DSDP files. Compare NOAA taxa with the LIMS approved taxa that the PIs have already approved in order create a list of unapproved NOAA taxa. Add PBDB data to unapproved taxa.

In [5]:
import sys
import csv
import glob
import os
import requests
import re
import time 

sys.path.append('../../')
import pandas as pd
import numpy as np

# import db 
import scripts.normalize_taxa as nt
from config import OUTPUT_DIR, CLEAN_DATA_DIR
from scripts.pbdb import get_parent_taxa, PBDB_TAXA_NAME

In [27]:
date='2021-11-29'

base_dir = CLEAN_DATA_DIR

input_file = f'../../raw_data/PI_processed_files/NOAA_taxa_lists_taxa_list_{date}.csv'
input_pbdb_file = f"../../output/taxa/NOAA/PI_normalized_taxa_list_with_pbdb_{date}.csv"


metadata_path = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_dsdp_files.csv'
LIMS_taxa_path = OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_2021-07-28.csv'


merged_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_merged_{date}.csv'
merged2_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_merged2_{date}.csv'

all_taxa_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'all_taxa_list_{date}.csv'

taxa_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_list_{date}.csv'
genus_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'genus_pbdb_{date}.csv'
taxa_pbdb_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_list_pbdb_{date}.csv'

In [28]:
def log_df(df, row_count=5):
    print(df.shape)
    return df.head(row_count)


## Create NOAA taxa list

In [5]:
metadata = pd.read_csv(metadata_path)
log_df(metadata)

(4477, 5)


,path,type,taxon_group,expedition,site
0,NOAA/DSDP_core_data/61/462/radiolar.csv,taxa,radiolarians,61,462
1,NOAA/DSDP_core_data/61/462/ageprof.csv,age,NaN,61,462
2,NOAA/DSDP_core_data/61/462/b_forams.csv,taxa,benthic_foraminfera,61,462
3,NOAA/DSDP_core_data/61/462/p_forams.csv,taxa,planktic_foraminfera,61,462
4,NOAA/DSDP_core_data/61/462/hr_desc.csv,hard_rock,NaN,61,462


read all the taxa files to get unique taxa names

In [6]:
# 9933
taxa = set()

for index, row in metadata.iterrows():
    if row['type'] == 'taxa':
        df = pd.read_csv(base_dir/row['path'])
        df.dropna(axis=0, inplace=True, how='all')
        df['temp'] = df['fossil'].str.strip() +  '|' + row['taxon_group']
        
        taxa.update(df['temp'])
        
len(taxa)

9933

In [7]:
list(taxa)[0:20]

[nan,
 'Amaurolithus delicatus|nannofossils',
 'Nonion labradorica|benthic_foraminfera',
 'Pullenia trintatensis|benthic_foraminfera',
 'Anomalinoides spissiformis|benthic_foraminfera',
 'Marginulinopsis mammilligera|benthic_foraminfera',
 'Chrysalogonium lanceolum|benthic_foraminfera',
 'Micrantholithus aequalis|nannofossils',
 'Apectodinium sumissum|dinoflagellates',
 'Parhabdolithus embergeri|nannofossils',
 'Reophax sabulosa|benthic_foraminfera',
 'Dictyocha fibula pumila|silicoflagellates',
 'Thalassiosira kozlovii|diatoms',
 'Stylodictya targaeformis|radiolarians',
 'Plectofrondicularia advena|benthic_foraminfera',
 'Globigerinatheka mexicana kugleri|planktic_foraminfera',
 'Biorbulina bilobata|planktic_foraminfera',
 'Proteacidites incurvatus|pollen',
 'Fursenkoina ciperana|benthic_foraminfera',
 'Siphonodosaria gracilima (q)|benthic_foraminfera']

In [8]:
# 9932
taxa_list = []

for taxon in taxa:
    if not pd.isna(taxon):
        taxon_name, taxon_group = taxon.split('|')
        
        # remove (text) (q) from taxon name    
        simplified_name = re.sub('\(.*?\)$', '', taxon_name).strip()
        taxon_name_parts = simplified_name.split(' ')

        data = {'verbatim_name': taxon_name, 
                'taxon_group': taxon_group, 
                'genus name': taxon_name_parts[0],
                'simplified_name': simplified_name}
        if len(taxon_name_parts) > 1:
            data['species name'] = taxon_name_parts[1]
        if len(taxon_name_parts) == 3:
            data['subspecies name'] = taxon_name_parts[2]

        taxa_list.append(data)
        
len(taxa_list)

9932

In [28]:
noaa_taxa_df = pd.DataFrame(taxa_list).sort_values('verbatim_name')
log_df(noaa_taxa_df)

(9932, 6)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name
8010,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN
5256,Abathomphalus intermedius,planktic_foraminfera,Abathomphalus,Abathomphalus intermedius,intermedius,NaN
96,Abathomphalus mayaroensis,planktic_foraminfera,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN
4463,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN
4709,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN


## compare and replace taxon groups 

replace NOAA taxon groups with LIMS taxon groups

In [29]:
LIMS_taxa_df = pd.read_csv(LIMS_taxa_path)

In [32]:
LIMS_groups = list(LIMS_taxa_df['taxon_group'].unique())
LIMS_groups.sort()
LIMS_groups

['benthic_forams',
 'bolboformids',
 'chrysophyte_cysts',
 'diatoms',
 'dinoflagellates',
 'ebridians',
 'nannofossils',
 'ostracods',
 'palynology',
 'planktic_forams',
 'radiolarians',
 'silicoflagellates']

In [33]:
noaa_groups = list(noaa_taxa_df['taxon_group'].unique())
noaa_groups.sort()
noaa_groups

['benthic_foraminfera',
 'diatoms',
 'dinoflagellates',
 'ebridians',
 'nannofossils',
 'ostracods',
 'phytoliths',
 'planktic_foraminfera',
 'pollen',
 'radiolarians',
 'silicoflagellates']

In [35]:
set(noaa_groups) -  set(LIMS_groups)

{'benthic_foraminfera', 'phytoliths', 'planktic_foraminfera', 'pollen'}

In [37]:
noaa_taxa_df = noaa_taxa_df.replace(['benthic_foraminfera', 'planktic_foraminfera'],
                                    ['benthic_forams', 'planktic_forams'])
noaa_taxa_df.head()

,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name
8010,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN
5256,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN
96,Abathomphalus mayaroensis,planktic_forams,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN
4463,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN
4709,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN


In [38]:
noaa_taxa_df.to_csv(all_taxa_path, index=False)

## create csv that compares NOAA taxa list with LIMS approved taxa

get NOAA taxa

In [39]:
# 9932
noaa_taxa_df = pd.read_csv(all_taxa_path)

log_df(noaa_taxa_df)

(9932, 6)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN
2,Abathomphalus mayaroensis,planktic_forams,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN
3,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN
4,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN


get LIMS taxa

In [40]:
# 4209
LIMS_taxa_df = pd.read_csv(LIMS_taxa_path)
log_df(LIMS_taxa_df)

(4209, 12)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams


add simplified_name without descriptor

In [41]:
del LIMS_taxa_df['normalized_name'] 


In [42]:
LIMS_taxa_df = nt.add_normalized_name_column(LIMS_taxa_df, 
                                                 include_descriptor=False, 
                                                 col_name="simplified_name")

In [43]:
LIMS_taxa_df[LIMS_taxa_df['non-taxa descriptor'].notna()].head(2)

,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,taxon_group,simplified_name
28,NaN,NaN,Globigerinoides,NaN,NaN,NaN,sacculifer,NaN,NaN,without sac,planktic_forams,Globigerinoides sacculifer
201,NaN,NaN,Bolivina,NaN,NaN,cf.,crenulata,NaN,NaN,crenulate,benthic_forams,Bolivina cf. crenulata


In [44]:
LIMS_taxa_df = pd.DataFrame(LIMS_taxa_df[['taxon_group', 'simplified_name']])
log_df(LIMS_taxa_df)

(4209, 2)


,taxon_group,simplified_name
0,benthic_forams,Euuvigerina miozea
1,benthic_forams,Euuvigerina rodleyi
2,benthic_forams,Foraminifera indet.
3,benthic_forams,Pleurostomellidae indet.
4,benthic_forams,Ostracoda indet.


### merge NOAA taxa with LIMS taxa

In [45]:
# 10109 
merged_df = pd.merge(noaa_taxa_df, LIMS_taxa_df,  
                     on=['simplified_name', 'taxon_group' ], 
                     how='left',
                     indicator='_merge_approved')

log_df(merged_df)


(10109, 7)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only
2,Abathomphalus mayaroensis,planktic_forams,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN,both
3,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only
4,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only


In [46]:
# 9932
merged_df = merged_df.drop_duplicates()
log_df(merged_df)

(9932, 7)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only
2,Abathomphalus mayaroensis,planktic_forams,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN,both
3,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only
4,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only


In [47]:
merged_df.to_csv(merged_path, index=False)


### compare merge methods
do merge on simplified_name without taxon groups

In [48]:
# 10114 
merged2_df = pd.merge(noaa_taxa_df, LIMS_taxa_df,  
                     on=['simplified_name'], 
                     how='left',
                     indicator='_merge_approved')

log_df(merged2_df)


(10114, 8)


,verbatim_name,taxon_group_x,genus name,simplified_name,species name,subspecies name,taxon_group_y,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,NaN,left_only
2,Abathomphalus mayaroensis,planktic_forams,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN,planktic_forams,both
3,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,NaN,left_only
4,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,NaN,left_only


In [49]:
# 9937
merged2_df = merged2_df.drop_duplicates()
log_df(merged2_df)

(9937, 8)


,verbatim_name,taxon_group_x,genus name,simplified_name,species name,subspecies name,taxon_group_y,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,NaN,left_only
2,Abathomphalus mayaroensis,planktic_forams,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN,planktic_forams,both
3,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,NaN,left_only
4,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,NaN,left_only


the reason for the count difference is because the LIMS taxa list sometimes puts a taxa in two groups

NOAA: Selenopemphix nephroides - dinoflagellates 
LIMS: Selenopemphix nephroides - dinoflagellates, palynology

## Created taxa list with unapproved NOAA taxa

Select unapproved NOAA taxa. When _merged is both, it means the taxa has been approved. When _merged is left_only, it means the taxa has not been approved.

In [51]:
merged_df = pd.read_csv(merged_path)
log_df(merged_df)

(9932, 7)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only
2,Abathomphalus mayaroensis,planktic_forams,Abathomphalus,Abathomphalus mayaroensis,mayaroensis,NaN,both
3,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only
4,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only


In [52]:
# (7763, 7)
LIMS_taxa_df = merged_df[merged_df['_merge_approved'] == 'left_only'].copy()

log_df(LIMS_taxa_df)

(7763, 7)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only
3,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only
4,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only
5,Abyssamina incisa,benthic_forams,Abyssamina,Abyssamina incisa,incisa,NaN,left_only


create csv of unapproved NOAA taxa

In [53]:
LIMS_taxa_df.to_csv(taxa_path, index=False)

## Create genus list with PBDB data for NOAA taxa that aren't approved

Look up the genus for unapproved taxa in PBDB

In [10]:
taxa_df = pd.read_csv(taxa_path)
log_df(taxa_df)

(7763, 7)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only
2,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only
3,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only
4,Abyssamina incisa,benthic_forams,Abyssamina,Abyssamina incisa,incisa,NaN,left_only


create a dataframe of unique genera

In [11]:
genus_df = pd.DataFrame(taxa_df['genus name'].unique(), columns=['genus name'])

log_df(genus_df)

(1707, 1)


,genus name
0,Abas
1,Abathomphalus
2,Abies
3,Abutilon
4,Abyssamina


add pbdb taxa data

In [95]:
for index, row in genus_df.iterrows():        
    if index < 250:
        continue
    time.sleep(0.5)
    
    if index % 50 == 0:
        print(index, end=' ')

        
    url =  PBDB_TAXA_NAME +  row['genus name']
        
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            genus_df.at[index, f'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            genus_df.at[index, f'pbdb_taxon_name'] = data[0]["taxon_name"]
            genus_df.at[index, f'pbdb_taxon_rank'] = data[0]["taxon_rank"]
            
            round = 0
            get_parent_taxa(genus_df, data[0]["parent_no"], data[0]["taxon_rank"], round, index, None)

           

250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 

In [96]:
genus_df.head()

,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,Abas,441243,Abas,genus,387088,Hemiaulaceae,426784,Hemiaulales,69587,Bacillariophyceae,432613,Ochrophyta,NaN,NaN
1,Abathomphalus,758,Abathomphalus,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
2,Abies,55065,Abies,genus,54794,Pinaceae,82140,Pinales,82141,Pinopsida,82139,Pinophyta,54311,Plantae
3,Abutilon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abyssamina,762,Abyssamina,genus,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria


create genus csv

In [97]:
genus_df.to_csv(genus_path, index=False)

# update genus file
look for genus that don't have higher pbdb data

In [33]:
genus_df = pd.read_csv(genus_path, dtype=str)

In [34]:
missing_df = genus_df[genus_df['pbdb_taxon_id'].notna() & 
         genus_df['family_taxon_id'].isna() &
         genus_df['order_taxon_id'].isna() &
         genus_df['class_taxon_id'].isna() &
         genus_df['phylum_taxon_id'].isna() &
         genus_df['kingdom_taxon_id'].isna()]

missing_df.shape

(30, 14)

In [37]:
for index, row in missing_df.iterrows():        
    time.sleep(0.5)
    
    
    print(index, row['genus name'])

        
    url =  PBDB_TAXA_NAME +  row['genus name']
        
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            genus_df.at[index, f'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            genus_df.at[index, f'pbdb_taxon_name'] = data[0]["taxon_name"]
            genus_df.at[index, f'pbdb_taxon_rank'] = data[0]["taxon_rank"]
            
            round = 0
            get_parent_taxa(genus_df, data[0]["parent_no"], data[0]["taxon_rank"], round, index, None)


7 Acaciapollenites
94 Anthocorys
151 Australopollis
319 Chenopodipollis
329 Chrysophyta
370 Comasphaeridium
405 Crassosphaera
443 Cyclopsiella
446 Cymatiosphaera
507 Dictyosphaeridium
553 Echitricolporites
746 Helminthopsis
779 Homotryblium
792 Hypericum
861 Leiofusa
862 Leiosphaeridia
974 Membranosphaera
981 Micrhystridium
993 Mimosaceae
1182 Platycystidia
1379 Sagittaria
1396 Schizocolpus
1409 Sethamphora
1458 Spondias
1509 Striatricolpites
1535 Tasmanites
1540 Tectatodinium
1637 Tuberculodinium
1675 Veryhachium
1678 Virola


In [39]:
genus_df.to_csv(genus_path, index=False)

## create taxa list with pbdb info for the PIs

In [61]:
genus_df = pd.read_csv(genus_path, dtype= str)
log_df(genus_df)

(1707, 14)


,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,Abas,441243,Abas,genus,387088,Hemiaulaceae,426784,Hemiaulales,69587,Bacillariophyceae,432613,Ochrophyta,NaN,NaN
1,Abathomphalus,758,Abathomphalus,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
2,Abies,55065,Abies,genus,54794,Pinaceae,82140,Pinales,82141,Pinopsida,82139,Pinophyta,54311,Plantae
3,Abutilon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abyssamina,762,Abyssamina,genus,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria


In [62]:
unapproved_df = pd.read_csv(taxa_path)

log_df(unapproved_df)

(7763, 7)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only
2,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only
3,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only
4,Abyssamina incisa,benthic_forams,Abyssamina,Abyssamina incisa,incisa,NaN,left_only


merge NOAA unapproved taxa with pbdb data

In [63]:
merged_df = pd.merge(LIMS_taxa_df, genus_df, 
                     on = 'genus name', 
                     how='left',
                     indicator='_merge_pbdb')

log_df(merged_df)

(7763, 21)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,...,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,_merge_pbdb
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only,441243,Abas,genus,...,Hemiaulaceae,426784,Hemiaulales,69587,Bacillariophyceae,432613,Ochrophyta,NaN,NaN,both
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only,758,Abathomphalus,genus,...,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,both
2,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only,55065,Abies,genus,...,Pinaceae,82140,Pinales,82141,Pinopsida,82139,Pinophyta,54311,Plantae,both
3,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,Abyssamina incisa,benthic_forams,Abyssamina,Abyssamina incisa,incisa,NaN,left_only,762,Abyssamina,genus,...,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,both


add columns 

In [64]:
merged_df['name'] = merged_df['verbatim_name'].str.strip()
merged_df['Comment'] = np.nan
merged_df['Notes (change to Internal only notes?)'] = np.nan
merged_df['Any taxon above genus'] = np.nan
merged_df['genus modifier'] = ''
merged_df['subgenera modifier'] = np.nan
merged_df['subgenera name'] = np.nan
merged_df['species modifier'] = np.nan
merged_df['subspecies modifier'] = np.nan
merged_df['non-taxa descriptor'] = np.nan
merged_df['comments'] = np.nan


remove (q) from name, and add (q) to genus modifier

In [65]:
for index, row in merged_df.iterrows():
    if '(q)' in row['name']:
        
        merged_df.at[index,'name']=re.sub('(.*?) \(q\)', r'? \1', row['name'])
        merged_df.at[index,'genus modifier'] = '?'
        
log_df(merged_df)

(7763, 32)


,verbatim_name,taxon_group,genus name,simplified_name,species name,subspecies name,_merge_approved,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,...,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,subgenera modifier,subgenera name,species modifier,subspecies modifier,non-taxa descriptor,comments
0,Abas wittii,diatoms,Abas,Abas wittii,wittii,NaN,left_only,441243,Abas,genus,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
1,Abathomphalus intermedius,planktic_forams,Abathomphalus,Abathomphalus intermedius,intermedius,NaN,left_only,758,Abathomphalus,genus,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
2,Abies sp.,pollen,Abies,Abies sp.,sp.,NaN,left_only,55065,Abies,genus,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,Abutilon sp. (q),pollen,Abutilon,Abutilon sp.,sp.,NaN,left_only,NaN,NaN,NaN,...,NaN,NaN,NaN,?,NaN,NaN,NaN,NaN,NaN,NaN
4,Abyssamina incisa,benthic_forams,Abyssamina,Abyssamina incisa,incisa,NaN,left_only,762,Abyssamina,genus,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


reorder columns and sort rows

In [66]:
merged_df = merged_df.reindex(columns=['taxon_group', 'verbatim_name', 'name', 'Comment',
                                       'Notes (change to Internal only notes?)',
                                       'Any taxon above genus', 
                                       'genus modifier', 'genus name', 
                                       'subgenera modifier', 'subgenera name',
                                       'species modifier', 'species name', 
                                       'subspecies modifier', 'subspecies name',
                                       'non-taxa descriptor', 'comments',
                                       'pbdb_taxon_id', 'pbdb_taxon_name',
                                       'pbdb_taxon_rank', '_simplified_name',
                                       '_merge_approved', '_merge_pbdb'
                                      ])

merged_df.sort_values(by=['taxon_group', 'verbatim_name'], inplace=True)

log_df(merged_df)

(7763, 22)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,...,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,_simplified_name,_merge_approved,_merge_pbdb
4,benthic_forams,Abyssamina incisa,Abyssamina incisa,NaN,NaN,NaN,,Abyssamina,NaN,NaN,...,NaN,NaN,NaN,NaN,762,Abyssamina,genus,NaN,left_only,both
131,benthic_forams,Adercotryma glomeratum,Adercotryma glomeratum,NaN,NaN,NaN,,Adercotryma,NaN,NaN,...,NaN,NaN,NaN,NaN,774,Adercotryma,genus,NaN,left_only,both
132,benthic_forams,Adercotryma sp.,Adercotryma sp.,NaN,NaN,NaN,,Adercotryma,NaN,NaN,...,NaN,NaN,NaN,NaN,774,Adercotryma,genus,NaN,left_only,both
144,benthic_forams,Alabamina decorata,Alabamina decorata,NaN,NaN,NaN,,Alabamina,NaN,NaN,...,NaN,NaN,NaN,NaN,788,Alabamina,genus,NaN,left_only,both
145,benthic_forams,Alabamina haitiensis,Alabamina haitiensis,NaN,NaN,NaN,,Alabamina,NaN,NaN,...,NaN,NaN,NaN,NaN,788,Alabamina,genus,NaN,left_only,both


In [67]:
merged_df.to_csv(taxa_pbdb_path, index=False)

## fix taxalist from PIs

update google sheet taxa list from PIs to deal with (q) in verbatim name. only need to do this once.

In [7]:
taxa_df = pd.read_csv(input_file, dtype=str)
log_df(taxa_df)

(7763, 20)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,Unnamed: 19
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,NaN,NaN,incisa,NaN,NaN,NaN,NaN,762,Abyssamina,genus,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,NaN,NaN,glomeratum,NaN,NaN,NaN,NaN,774,Adercotryma,genus,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,NaN,NaN,sp.,NaN,NaN,NaN,NaN,774,Adercotryma,genus,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,NaN,NaN,decorata,NaN,NaN,NaN,NaN,788,Alabamina,genus,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,NaN,NaN,haitiensis,NaN,NaN,NaN,NaN,788,Alabamina,genus,NaN


In [8]:

for index, row in taxa_df.iterrows():
    # don't overwrite existing name
    if isinstance(row['name'], str) :
        continue
    # if verbatim name has '(q)'
    if '(q)' in row['verbatim_name']:
        # set 'name' to 'verbatim name' without '(q)' 
        taxa_df.at[index,'name']=re.sub('(.*?) ?\(q\)', r'? \1', row['verbatim_name'])
        # set 'genus modifier' to '?'
        taxa_df.at[index,'genus modifier'] = '?'

        
log_df(taxa_df)

(7763, 20)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,Unnamed: 19
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,NaN,NaN,incisa,NaN,NaN,NaN,NaN,762,Abyssamina,genus,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,NaN,NaN,glomeratum,NaN,NaN,NaN,NaN,774,Adercotryma,genus,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,NaN,NaN,sp.,NaN,NaN,NaN,NaN,774,Adercotryma,genus,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,NaN,NaN,decorata,NaN,NaN,NaN,NaN,788,Alabamina,genus,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,NaN,NaN,haitiensis,NaN,NaN,NaN,NaN,788,Alabamina,genus,NaN


In [112]:
taxa_df.to_csv('cleaned_data/taxa/draft/NOAA/google_sheet_taxa_lists_2021-08-05.csv', index=False)